# Test Model

In [3]:
from keras.datasets import reuters
from keras import models, layers
import numpy as np
import matplotlib.pyplot as plt
from kapre.time_frequency import Melspectrogram

### Load Data

In [4]:
data_path='../data/data_15000.npy'
labels_path = '../data/labels_15000.npy'

In [5]:
data = np.load(data_path)
labels = np.load(labels_path)

In [6]:
#add "channel dimension"
#audio data is always mono
data = np.expand_dims(data, axis=1)

### Split Data

In [8]:
train_data = data[:7500]
train_labels = labels[:7500]

val_data = data[7500:11250]
val_labels = labels[7500:11250]

test_data = data[11250:]
test_labels = labels[11250:]

### Define Model

Similar Architecture to [paper](https://github.com/twerkmeister/iLID/blob/master/Deep%20Audio%20Paper%20Thomas%20Werkmeister%2C%20Tom%20Herold.pdf)

In [10]:
model = models.Sequential()
model.add(Melspectrogram(n_dft=512, input_shape=(1, 5 * 16000,),
                         padding='same', sr=16000, n_mels=28,
                         fmin=0.0, fmax=10000, power_melgram=1.0,
                         return_decibel_melgram=False, trainable_fb=False,
                         trainable_kernel=False))
model.add(layers.Conv2D(64, (3, 6), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 6), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 6), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
melspectrogram_2 (Melspectro (None, 28, 313, 1)        270364    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 26, 308, 64)       1216      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 154, 64)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 149, 64)       73792     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 74, 64)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 69, 128)        147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 1, 34, 128)        0         
__________

In [12]:
model.compile(optimizer='Rmsprop',
              metrics=['accuracy'],
              loss='categorical_crossentropy')

### Train Model

In [13]:
history = model.fit(train_data,
                    train_labels,
                    batch_size=64,
                    epochs=9,
                    validation_data=(val_data, val_labels))

Train on 7500 samples, validate on 3750 samples
Epoch 1/9
7500/7500 [==============================] - 17s 2ms/step - loss: 1.0368 - acc: 0.4888 - val_loss: 1.0159 - val_acc: 0.5152
Epoch 2/9
7500/7500 [==============================] - 13s 2ms/step - loss: 0.8200 - acc: 0.6319 - val_loss: 1.1726 - val_acc: 0.4509
Epoch 3/9
7500/7500 [==============================] - 13s 2ms/step - loss: 0.6812 - acc: 0.7092 - val_loss: 0.9732 - val_acc: 0.6037
Epoch 4/9
7500/7500 [==============================] - 14s 2ms/step - loss: 0.5754 - acc: 0.7692 - val_loss: 0.7841 - val_acc: 0.6461
Epoch 5/9
7500/7500 [==============================] - 14s 2ms/step - loss: 0.4997 - acc: 0.8020 - val_loss: 0.5292 - val_acc: 0.7917
Epoch 6/9
7500/7500 [==============================] - 13s 2ms/step - loss: 0.4182 - acc: 0.8345 - val_loss: 0.8646 - val_acc: 0.6987
Epoch 7/9
7500/7500 [==============================] - 13s 2ms/step - loss: 0.3441 - acc: 0.8629 - val_loss: 0.6470 - val_acc: 0.7973
Epoch 8/9
7500

### Save Model
78% Accuracy Test is not bad!!

In [15]:
model.save('test.h5')